In [1]:
from gurobipy import quicksum, GRB, Model


In [2]:
I = 6

capa = (80,90,80,90,80,90)
reservations = (111,159,104,148,170,145)

In [3]:
m = Model("Overbooked airline")
x = m.addVars(I, I, vtype=GRB.CONTINUOUS, name='x', lb = 0)
y = m.addVars(I, vtype=GRB.CONTINUOUS, name='y', lb = 0)


m.setObjective(
    (50+3*10) * quicksum(y[i] for i in range(I))
    + 40*quicksum(x[i,j] for i in range(I) for j in range(i+1,I))
    + 10*quicksum((j-i-1)* x[i,j] for i in range(I) for j in range(i+1, I))
)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-10-26


In [4]:
# serve all customers:

m.addConstrs(
    quicksum(x[i,j] for j in range(I)) + y[i] == reservations[i] for i in range(I)
)

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>}

In [5]:
# capacity constrains

m.addConstrs(
    quicksum(x[i,j] for i in range(I)) <= capa[j] for j in range(I)
)

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>}

In [6]:
# cant book reservations on earlier flights:

m.addConstrs(
    x[i,j] == 0 for j in range(I) for i in range(j+1, I)
)

{(0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 4): <gurobi.Constr *Awaiting Model Update*>,
 (0, 5): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 5): <gurobi.Constr *Awaiting Model Update*>,
 (2, 3): <gurobi.Constr *Awaiting Model Update*>,
 (2, 4): <gurobi.Constr *Awaiting Model Update*>,
 (2, 5): <gurobi.Constr *Awaiting Model Update*>,
 (3, 4): <gurobi.Constr *Awaiting Model Update*>,
 (3, 5): <gurobi.Constr *Awaiting Model Update*>,
 (4, 5): <gurobi.Constr *Awaiting Model Update*>}

In [7]:
m.optimize()

print(m.getObjective())


Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 27 rows, 42 columns and 93 nonzeros
Model fingerprint: 0xb444ecb2
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e+01, 8e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e+01, 2e+02]
Presolve removed 18 rows and 22 columns
Presolve time: 0.01s
Presolved: 9 rows, 20 columns, 34 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.4000000e+03   6.408200e+01   0.000000e+00      0s
      13    2.6160000e+04   0.000000e+00   0.000000e+00      0s

Solved in 13 iterations and 0.01 seconds (0.00 work units)
Optimal objective  2.616000000e+04
<gurobi.LinExpr: 40.0 x[0,1] + 50.0 x[0,2] + 60.0 x[0,3] + 70.0 x[0,4] + 80.0 x[0,5] + 40.0 x[1,2] + 50.0 x[1,3] + 60.0 x[1,4] + 70.0 x[1,5] + 40.0 x[2,3] + 50.0 x[2,4] + 60.0 x[2,5] + 40.0 x[3,4] + 50.0 x[3,5] + 40.0 x

In [8]:
for j in range(I):
    guests = quicksum(x[i,j] for i in range(0,j+1)).getValue()
    print(f"# guests in flight {j}: {guests}")

another_airline = quicksum(y[i] for i in range(I)).getValue()

print("Booking of flights from another airline:", another_airline)

print(f"Total guests: {another_airline + quicksum(x[i,j] for i in range(I) for j in range(I)).getValue()}")
print("Total reservations = ", sum(reservations))

print("Total costs = ", m.getObjective().getValue())


# guests in flight 0: 80.0
# guests in flight 1: 90.0
# guests in flight 2: 80.0
# guests in flight 3: 90.0
# guests in flight 4: 80.0
# guests in flight 5: 90.0
Booking of flights from another airline: 327.0
Total guests: 837.0
Total reservations =  837
Total costs =  26160.0
